In [ ]:
# Import necessary libraries and authenticate
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Counters for successes and failures
success_count = 0
failure_count = 0

def get_existing_items_in_folder(folder_id):
    """Retrieve a list of file and folder names in a specified folder."""
    file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
    return {item['title']: item['id'] for item in file_list}

def create_folder(parent_folder_id, folder_name):
    """Creates a folder in Google Drive and returns its ID."""
    folder_metadata = {
        'title': folder_name,
        'mimeType': 'application/vnd.google-apps.folder',
        'parents': [{'id': parent_folder_id}]
    }
    folder = drive.CreateFile(folder_metadata)
    folder.Upload()
    return folder['id']

def copy_file(file_id, destination_folder_id, existing_files):
    """Copies a file to a specified folder, only if it hasn't been copied yet."""
    global success_count, failure_count
    try:
        original_file = drive.CreateFile({'id': file_id})
        original_file.FetchMetadata()  # Fetch file metadata

        if original_file['title'] in existing_files:
            print(f"File '{original_file['title']}' already exists in the destination folder. Skipping.")
            return None

        # Create a new file with the same metadata
        copied_file = drive.CreateFile({
            'title': original_file['title'],
            'parents': [{'id': destination_folder_id}]
        })

        # Check the file's MIME type
        mime_type = original_file['mimeType']

        # Handle Google Apps file types (Docs, Sheets, Slides)
        if mime_type == 'application/vnd.google-apps.document':
            export_mime_type = 'application/pdf'  # Export Google Docs as PDF
            original_file.GetContentFile(original_file['title'] + '.pdf', mimetype=export_mime_type)
            copied_file.SetContentFile(original_file['title'] + '.pdf')
            os.remove(original_file['title'] + '.pdf')  # Clean up local file
        elif mime_type == 'application/vnd.google-apps.spreadsheet':
            export_mime_type = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'  # Export Sheets as Excel
            original_file.GetContentFile(original_file['title'] + '.xlsx', mimetype=export_mime_type)
            copied_file.SetContentFile(original_file['title'] + '.xlsx')
            os.remove(original_file['title'] + '.xlsx')  # Clean up local file
        elif mime_type == 'application/vnd.google-apps.presentation':
            export_mime_type = 'application/pdf'  # Export Slides as PDF
            original_file.GetContentFile(original_file['title'] + '.pdf', mimetype=export_mime_type)
            copied_file.SetContentFile(original_file['title'] + '.pdf')
            os.remove(original_file['title'] + '.pdf')  # Clean up local file
        else:
            # For non-Google Apps binary files (e.g., images, videos), handle as raw content
            if 'google-apps' not in mime_type:
                local_filename = original_file['title']  # Temporary local file to store binary content
                original_file.GetContentFile(local_filename)  # Download original file content to a local file
                copied_file.SetContentFile(local_filename)  # Set content of new file using the downloaded file
                os.remove(local_filename)  # Clean up by deleting the local file after copying

        copied_file.Upload()  # Upload the copied file
        print(f"Successfully copied file: {original_file['title']}")
        success_count += 1
        return copied_file['id']
    except Exception as e:
        print(f"Failed to copy file: {original_file['title']}. Error: {e}")
        failure_count += 1
        return None

def copy_folder(source_folder_id, destination_folder_id):
    """Recursively copies a folder and its contents, only copying items that don't already exist."""
    global success_count, failure_count
    try:
        folder_to_copy = drive.CreateFile({'id': source_folder_id})
        folder_to_copy.FetchMetadata()

        # Create a new folder inside the destination folder
        existing_items = get_existing_items_in_folder(destination_folder_id)
        if folder_to_copy['title'] in existing_items:
            print(f"Folder '{folder_to_copy['title']}' already exists in the destination folder. Skipping.")
            new_folder_id = existing_items[folder_to_copy['title']]
        else:
            new_folder_id = create_folder(destination_folder_id, folder_to_copy['title'])
            print(f"Successfully copied folder: {folder_to_copy['title']}")

        # List all contents of the source folder
        file_list = drive.ListFile({'q': f"'{source_folder_id}' in parents and trashed=false"}).GetList()

        for item in file_list:
            if item['mimeType'] == 'application/vnd.google-apps.folder':
                # Recursively copy the folder
                copy_folder(item['id'], new_folder_id)
            else:
                # Copy the file
                copy_file(item['id'], new_folder_id, get_existing_items_in_folder(new_folder_id))
    except Exception as e:
        print(f"Failed to copy folder: {folder_to_copy['title']}. Error: {e}")
        failure_count += 1

# Find the folder "NULab and DITI" shared with you
folder_name_to_copy = "NULab and DITI"
file_list = drive.ListFile({'q': "sharedWithMe = true and trashed=false"}).GetList()

source_folder_id = None
for item in file_list:
    if item['title'] == folder_name_to_copy and item['mimeType'] == 'application/vnd.google-apps.folder':
        source_folder_id = item['id']
        break

if source_folder_id:
    # Create the "Copy of NULab and DITI" folder or find its ID if it already exists
    root_folder_id = 'root'  # Google Drive's root folder ID
    existing_root_items = get_existing_items_in_folder(root_folder_id)
    if "Copy of NULab and DITI" in existing_root_items:
        copy_folder_id = existing_root_items["Copy of NULab and DITI"]
        print(f"Folder 'Copy of NULab and DITI' already exists. Using existing folder.")
    else:
        copy_folder_id = create_folder(root_folder_id, "Copy of NULab and DITI")

    # Copy the "NULab and DITI" folder and its contents
    copy_folder(source_folder_id, copy_folder_id)

    # Print the final tally of successes and failures
    print("Copy process completed.")
    print(f"Total successes: {success_count}")
    print(f"Total failures: {failure_count}")
else:
    print(f"Folder '{folder_name_to_copy}' not found in your shared files.")
